In [1]:
%matplotlib inline

from matplotlib import pyplot as plt

from skimage import io
from skimage import exposure
from skimage import transform
from skimage import color

import numpy as np

import os
from os import path

import supervisely_lib as sly
from supervisely_lib.annotation.annotation import Annotation
from supervisely_lib.annotation.obj_class import ObjClass
from supervisely_lib.geometry.bitmap import Bitmap
from supervisely_lib.project.project_meta import ProjectMeta

In [2]:
# Change this field to the name of your team, where target workspace exists.
team_name = "marcwang17" # Automatically inserted

# Change this field to the of your workspace, where target project exists.
workspace_name = "First Workspace" # Automatically inserted

# If you are running this notebook on a Supervisely web instance, the connection
# Edit those values if you run this notebook on your own PC
# details below will be filled in from environment variables automatically.
#
# If you are running this notebook locally on your own machine, edit to fill in the
# connection details manually. You can find your access token at
# "Your name on the top right" -> "Account settings" -> "API token".
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']

In [3]:
# Initialize API object
api = sly.Api(address, token)

In [4]:
# Get IDs of team and workspace
team = api.team.get_info_by_name(team_name)
workspace = api.workspace.get_info_by_name(team.id, workspace_name)

In [5]:
src_proj_name = 'bimodal_powder_2'

src_project = api.project.get_info_by_name(workspace.id, src_proj_name)
if src_project is None:
    raise RuntimeError("project {!r} not found".format(src_proj_name))

In [6]:
meta_json = api.project.get_meta(src_project.id)
meta = sly.ProjectMeta.from_json(meta_json)
print(meta)

ProjectMeta:
Object Classes
+-------------+--------+----------------+
|     Name    | Shape  |     Color      |
+-------------+--------+----------------+
|   particle  | Bitmap | [177, 36, 25]  |
| particle_wh | Bitmap | [74, 144, 226] |
+-------------+--------+----------------+
Tags
+------+------------+-----------------+
| Name | Value type | Possible values |
+------+------------+-----------------+
+------+------------+-----------------+



In [7]:
target_proj_name = 'raw bimodal powder'

target_project = api.project.get_info_by_name(workspace.id, target_proj_name)
if target_project is None:
    target_project = api.project.create(workspace.id, target_proj_name, change_name_if_conflict=True)
    
    particle_meta = ObjClass('particle', Bitmap, [177,36,25])
    particle_h_meta = ObjClass('particle_wh', Bitmap, [74,144,226])
    
    meta = ProjectMeta.from_json(api.project.get_meta(target_project.id))
    meta = meta.add_obj_class(particle_meta)
    meta = meta.add_obj_class(particle_h_meta)
    
    api.project.update_meta(target_project.id, meta.to_json())
    

In [8]:
file_list = ["2020-03-04 Marc PM2 C4_5um scan_recon.tiff",
             "2020-03-04 Marc PM3 C4_5um scan_recon.tiff",
             "2020-03-04 Marc PM4 C4_5um scan_recon.tiff",
             "2020-03-11 Marc PM5 C1_5um scanA_recon.tiff",
             "2020-03-11 Marc PM4 C1_5um scan_recon.tiff"]

folder_path = path.join('.','data','process_map')

In [9]:
sampled_index = [[432,335,683],
                 [511,535,506],
                 [345,661,488],
                 [325,679,596],
                 [453,420,496]]

im_size = 150

In [10]:
sampled_list = []

for filename, index in zip(file_list, sampled_index):
    im = io.imread(path.join(folder_path, filename))
    
    sampled_im = im[index[2]:index[2] + im_size, 
                    index[0]:index[0] + im_size, 
                    index[1]]
    
    sampled_list.append(sampled_im)

In [11]:
def find_name(name):
    def func(x):
        if(x.name == name):
            return True
        else:
            return False
    return func

In [12]:
src_datasets = api.dataset.get_list(src_project.id)

target_dataset = api.dataset.create(target_project.id, 'trial', change_name_if_conflict=True)
list_image = api.image.get_list(src_datasets[0].id)
for i, im in enumerate(sampled_list):
    
    filter_f = find_name('smooth_upscaled_{}.png'.format(i))
    result_it = filter(filter_f,list_image)
    result = [j for j in result_it]
    remote_im = result[0]
    
    upsampled = transform.rescale(im, scale = 4, order = 0)
    amp_contrast = exposure.equalize_hist(upsampled)
    rescaled_uint8 = exposure.rescale_intensity(amp_contrast, out_range = np.uint8)
    
    rgb_im = color.gray2rgb(rescaled_uint8)
    
    im_info = api.image.upload_np(target_dataset.id, 'im_{}.png'.format(i), rgb_im)
    
    api.annotation.copy(remote_im.id, im_info.id)
    